In [49]:
import requests
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  KFold
import warnings
import re
warnings.filterwarnings('ignore')

In [50]:
movies_model_data_scaled = pd.read_csv("movies_db_5.csv", delimiter=",",engine='python') #Provient de 5_Scaling

In [51]:
movies_model_data_scaled.drop(["id",'vote_average','vote_count','revenue'],axis=1,inplace=True)

In [52]:
data_0 = movies_model_data_scaled[movies_model_data_scaled["target"] == 0]
data_1 = movies_model_data_scaled[movies_model_data_scaled["target"] == 1]
data_2 = movies_model_data_scaled[movies_model_data_scaled["target"] == 2]
# data_3 = movies_model_data_scaled[movies_model_data_scaled["target"] == "SUPER HIT"]

x_0 = data_0.drop(["target","title"],axis=1)
x_1 = data_1.drop(["target","title"],axis=1)
x_2 = data_2.drop(["target","title"],axis=1)
# x_3 = data_3.drop(["target","title"],axis=1)

y_0 = data_0["target"]
y_1 = data_1["target"]
y_2 = data_2["target"]
# y_3 = data_3["target"]

x_0_train, x_0_test, y_0_train, y_0_test = train_test_split(x_0, y_0, test_size=0.33)
x_1_train, x_1_test, y_1_train, y_1_test = train_test_split(x_1, y_1, test_size=0.33)
x_2_train, x_2_test, y_2_train, y_2_test = train_test_split(x_2, y_2, test_size=0.33)
# x_3_train, x_3_test, y_3_train, y_3_test = train_test_split(x_3, y_3, test_size=0.33)

x_train = pd.concat([x_0_train,x_1_train,x_2_train])
x_test = pd.concat([x_0_test,x_1_test,x_2_test])
y_train = pd.concat([y_0_train,y_1_train,y_2_train])
y_test = pd.concat([y_0_test,y_1_test,y_2_test])

"""
x_train = x_train.drop(["pop_company", "pop_actors", "pop_producers"],axis=1)
x_test = x_test.drop(["pop_company", "pop_actors", "pop_producers"],axis=1)
"""


'\nx_train = x_train.drop(["pop_company", "pop_actors", "pop_producers"],axis=1)\nx_test = x_test.drop(["pop_company", "pop_actors", "pop_producers"],axis=1)\n'

### LDA From Scratch


Subset the training set into two sets: class0 where Purchased = 0 and class1 where Purchased = 1)

In [53]:
x_0_train, x_1_train, x_2_train

(        budget   runtime  pop_actors  pop_directors  pop_producer
 1138  0.899032  1.460983    0.149775       0.053210      1.314602
 5684  1.270260  0.013735    1.599094       0.030340      0.363706
 656  -0.672496 -0.798135   -1.176014       1.353961      0.647789
 6467  0.280321 -0.233356    0.149214       0.084890     -0.331584
 3283 -0.689300 -0.233356   -0.923737      -0.396072     -0.662220
 ...        ...       ...         ...            ...           ...
 4082 -0.239397 -0.127460    0.573519      -0.661881     -0.058774
 4995  0.032836  1.037398    0.422921      -0.523470     -0.217387
 6566 -0.573502  0.119631    0.813791       1.813238      1.422190
 1310 -0.363140  0.154930   -0.857287      -0.149746     -0.603907
 6336 -0.313643  0.225528    1.058439      -0.471122     -0.470564
 
 [4205 rows x 5 columns],
         budget   runtime  pop_actors  pop_directors  pop_producer
 7061 -0.647747  0.402021    0.492849       0.526380     -0.219039
 60    1.245511  2.661140    1.064

Compute  π0  and π1

In [54]:
pi0 = len(x_0_train) / len(x_train)
pi1 = len(x_1_train) / len(x_train)
pi2 = len(x_2_train) / len(x_train)

In [55]:
pi0, pi1, pi2

(0.8850768259313828, 0.06988002525784046, 0.04504314881077668)

Compute  μ0 and  μ1

In [56]:
np.mean(x_0_train, axis=0)

budget          -0.126457
runtime          0.026226
pop_actors      -0.006533
pop_directors   -0.039393
pop_producer    -0.006941
dtype: float64

In [57]:
mu0 = np.mean(x_0_train, axis=0)
mu1 = np.mean(x_1_train, axis=0)
mu2 = np.mean(x_2_train, axis=0)

In [58]:
mu0, mu1, mu2

(budget          -0.126457
 runtime          0.026226
 pop_actors      -0.006533
 pop_directors   -0.039393
 pop_producer    -0.006941
 dtype: float64, budget           2.023590
 runtime          0.704293
 pop_actors       1.145735
 pop_directors    0.891018
 pop_producer     0.620920
 dtype: float64, budget          -0.682707
 runtime         -1.865674
 pop_actors      -1.608732
 pop_directors   -0.662021
 pop_producer    -0.592265
 dtype: float64)

Compute Σ

In [59]:
sigma = np.array(((len(x_0_train)-1) * x_0_train.cov() +
         (len(x_1_train)-1) * x_1_train.cov() + 
         (len(x_2_train)-1) * x_2_train.cov()) / (len(x_0_train) + len(x_1_train) + len(x_2_train) - 3))

In [60]:
sigma

array([[0.67224074, 0.1274441 , 0.3161503 , 0.10800756, 0.18933473],
       [0.1274441 , 0.76943876, 0.19573863, 0.12557806, 0.11157214],
       [0.3161503 , 0.19573863, 0.80356043, 0.14770044, 0.26309476],
       [0.10800756, 0.12557806, 0.14770044, 0.95691959, 0.33313894],
       [0.18933473, 0.11157214, 0.26309476, 0.33313894, 1.0141541 ]])

Compute Delta0 an Delta1

In [61]:
from math import *

In [62]:
sigmaI=np.linalg.inv(sigma)
delta0=np.dot(np.dot(x_test, sigmaI), mu0) - 0.5 * np.dot(np.dot(mu0.transpose(), sigmaI), mu0) + log(pi0)
delta1=np.dot(np.dot(x_test, sigmaI), mu1) - 0.5 * np.dot(np.dot(mu1.transpose(), sigmaI), mu1) + log(pi1)
delta2=np.dot(np.dot(x_test, sigmaI), mu2) - 0.5 * np.dot(np.dot(mu2.transpose(), sigmaI), mu2) + log(pi2)

In [63]:
delta0

array([-0.25087783, -0.1301839 , -0.44287651, ..., -0.28865288,
       -0.14812386, -0.10447857])

In [64]:
y_pred_lda_scratch = pd.DataFrame(np.array([delta0, delta1, delta2]).transpose())

In [65]:
y_pred_lda_scratch.columns = [0, 1, 2]

In [66]:
y_pred_lda_scratch

,0,1,2
0,-0.250878,-9.566710,2.480187
1,-0.130184,-6.244026,-6.233020
2,-0.442877,-2.291796,-5.781194
3,-0.025372,-5.322410,-9.776106
4,-0.284157,-4.160535,-7.062142
...,...,...,...
2337,-0.072036,-8.580026,-3.571930
2338,-0.264373,-9.639618,2.935230
2339,-0.288653,-9.794284,3.824989
2340,-0.148124,-8.983904,-1.096228


In [67]:
y_pred_lda_scratch["predictions"] = y_pred_lda_scratch.idxmax(axis="columns")

In [68]:
y_pred_lda_scratch

,0,1,2,predictions
0,-0.250878,-9.566710,2.480187,2
1,-0.130184,-6.244026,-6.233020,0
2,-0.442877,-2.291796,-5.781194,0
3,-0.025372,-5.322410,-9.776106,0
4,-0.284157,-4.160535,-7.062142,0
...,...,...,...,...
2337,-0.072036,-8.580026,-3.571930,0
2338,-0.264373,-9.639618,2.935230,2
2339,-0.288653,-9.794284,3.824989,2
2340,-0.148124,-8.983904,-1.096228,0


In [69]:
precision_score(y_test, y_pred_lda_scratch["predictions"], average="micro")

0.9005123825789924